In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from FoodSeg103_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prediction

Load the images as batches

In [3]:
img_nums = [1328]#[n for n in range(0, 4500, 1000)] #2750 #1234 #2314
imgs = load_food_batch(img_nums, load_gts=False)[0]
num_imgs = len(imgs)
print(f"Total images: {num_imgs}")

Total images: 1


Check and filter for resolution

In [22]:
resol = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}

# Check for images with resolution over a certain threshold
thresh = 640_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels
1000:   196,608 pixels
2000:   172,544 pixels
3000:   174,080 pixels
4000:   196,608 pixels


In [23]:
# Filter out those images
imgs = {im_num: im for im_num, im in imgs.items() if resol[im_num] <= thresh}
num_imgs = len(imgs)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 640,000: 5
   0: 196,608 pixels
1000: 196,608 pixels
2000: 172,544 pixels
3000: 174,080 pixels
4000: 196,608 pixels


In [4]:
# Where to find and save the data
folder_path = "./FoodSeg103_results/data/food_run06a_examples"

# Which scribbles to use
mode = "all"
bins = [0.1] #[0.05, 0.3, 1.5]
scribble_width = 2
# suff = ["TEST"]
s = "run06"

# Which prediction models to use
convpaint_layers = [[0], [0,2], [0,2,5]]
convpaint_scalings = [[1], [1,2], [1,2,4], [1,2,4,8]]
model = "vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
use_dino = True
use_ilastik = True
pred_seed = 123

save_res = False
show_res = True

Loop and predict

In [18]:
for bin in bins:
    for img_num, img in imgs.items():
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        for layers in [[0]]:#convpaint_layers:
            for scalings in [[1]]:#convpaint_scalings:
                print(f"   ConvPaint: layers {layers}, scalings {scalings}")
                pred_conv = pred_food_convpaint(img, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, 
                                                layer_list=layers, scalings=scalings, model=model, random_state=12,
                                                save_res=save_res, show_res=show_res, ground_truth=None)
        if use_ilastik:
            print("   Ilastik")
            pred_ila = pred_food_ilastik(img, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                         random_state=pred_seed,
                                         save_res=save_res, show_res=show_res, ground_truth=None)
        if use_dino:
            print("   DINO")
            pred_dino = pred_food_dino(img, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, 
                                            dinov2_model='s', upscale_order=0, random_state=pred_seed,
                                            save_res=save_res, show_res=show_res, ground_truth=None)
        print("\n")

IMG 1328: bin 0.1, suff run06
   ConvPaint: layers [0], scalings [1]


Show the last predictions (optionally with image, ground truth and scribbles)

In [ ]:
ground_truth = load_food_data(img_num, load_image=False)[1]
scribbles = get_food_img_data(folder_path, img_num, load_scribbles=True, mode=mode, bin=bin, scribble_width=scribble_width, suff=s)["scribbles"]

In [ ]:
print(f"Ilastik:   {np.sum(pred_ila == ground_truth) / np.prod(pred_ila.shape) * 100:.2f}%")
print(f"Convpaint: {np.sum(pred_conv == ground_truth) / np.prod(pred_conv.shape) * 100:.2f}%")
print(f"DINOv2:    {np.sum(pred_dino == ground_truth) / np.prod(pred_dino.shape) * 100:.2f}%")

In [ ]:
v = napari.Viewer()
v.add_image(img)
v.add_labels(ground_truth)
v.add_labels(scribbles)
v.add_labels(pred_conv)
v.add_labels(pred_ila)
v.add_labels(pred_dino)

## Test Computation Times

In [5]:
res_list = []
for img_num, img in imgs.items():
    print(f"IMG {img_num}:")
    labels = get_food_img_data(folder_path, img_num, load_scribbles=True, mode="all", bin=0.1, scribble_width=2, suff="run06", load_pred=False, pred_tag="convpaint")["scribbles"]
    for layers in convpaint_layers:
        for scalings in convpaint_scalings:
            pred_tag = generate_convpaint_tag(layers, scalings, model)
            print(pred_tag)
            t = time_food(img, labels, pred_type="convpaint", layer_list=layers, scalings=scalings, model_name=model)
            res = pd.DataFrame({"img": img_num, "prediction tag": pred_tag, "prediction type": "convpaint",
                                "layers": str(layers)[1:-1].replace(", ", "-"), "scalings": str(scalings)[1:-1].replace(", ", "-"),
                                "t_features": t[0], "t_pred": t[1], "t_tot": t[2]}, index=[0])
            res_list.append(res)
    if use_ilastik:
        print("ilastik")
        t = time_food(img, labels, pred_type="ilastik")
        res = pd.DataFrame({"img": img_num, "prediction tag": "ilastik", "prediction type": "classical filters",
                            "layers": "-", "scalings": "-",
                            "t_features": t[0], "t_pred": t[1], "t_tot": t[2]}, index=[0])
        res_list.append(res)
    if use_dino:
        print("DINOv2")
        t = time_food(img, labels, pred_type="dino")
        res = pd.DataFrame({"img": img_num, "prediction tag": "dino", "prediction type": "DINOv2",
                            "layers": "-", "scalings": "-",
                            "t_features": t[0], "t_pred": t[1], "t_tot": t[2]}, index=[0])
        res_list.append(res)

df = pd.concat(res_list)

IMG 1328:
convpaint_l-0_s-1
convpaint_l-0_s-1-2
convpaint_l-0_s-1-2-4
convpaint_l-0_s-1-2-4-8
convpaint_l-0-2_s-1
convpaint_l-0-2_s-1-2
convpaint_l-0-2_s-1-2-4
convpaint_l-0-2_s-1-2-4-8
convpaint_l-0-2-5_s-1
convpaint_l-0-2-5_s-1-2
convpaint_l-0-2-5_s-1-2-4
convpaint_l-0-2-5_s-1-2-4-8
ilastik
DINOv2


C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [6]:
df.groupby(["prediction type", "scalings", "layers"]).mean(True)[["t_features", "t_pred", "t_tot"]].round(2)

t_features  t_pred  t_tot
prediction type   scalings layers                           
DINOv2            -        -             0.70    0.49   1.19
classical filters -        -             0.44    0.66   1.10
convpaint         1        0             0.05    1.09   1.22
                           0-2           0.37    1.45   1.98
                           0-2-5         0.51    1.51   2.56
                  1-2      0             0.07    1.09   1.44
                           0-2           0.68    2.69   4.41
                           0-2-5         1.08    2.81   7.30
                  1-2-4    0             0.12    1.39   2.00
                           0-2           0.50    2.45   4.22
                           0-2-5         0.77    3.56   6.25
                  1-2-4-8  0             0.16    2.35   3.27
                           0-2           0.44    2.62   4.42
                           0-2-5         1.15    4.28   9.11

In [7]:
# df.to_csv(f"./food_time_prelim1.csv", index=False)